In [1]:
using Flux
using Flux.Data: DataLoader
using PyCall
using Plots
using PlotThemes
using TOML
using Pkg
using BSON: @save

In [2]:
env_path = TOML.parse(read("mltools.toml", String))["env_path"]
Pkg.activate(env_path)

  Activating environment at `C:\Users\nikhi\OneDrive\Documents\julia\MLTools\Project.toml`


In [3]:
import MLTools as mlt

In [4]:
df = mlt.getdf(joinpath(@__DIR__, "filtered_dataset.csv"))

,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME
0,COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=C(C=C3)B...,Active
1,COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=CC=C3)N)...,Active
2,C1CN(CCN1)C(=O)C2=C(SC(=C2)C3=CC=CC=C3)N,Active
3,C1=CC=C(C=C1)C(=O)C2=C(C(=C(N2)N)C(=O)N)C3=CC=...,Active
4,CC1=C(C(=O)OC(C1)[C@@H](C)C2CC[C@@H]3[C@@]2(CC...,Active
...,...,...
18229,C1CN(CCC1CNC(=O)NC2=CC=CC=C2)C3=CC=C(C=C3)S(=O...,Inactive
18230,CN1C(=O)C(=C(N(C1=O)CC2=CC=CC=C2)N)C(=O)CN3CCOCC3,Inactive
18231,COC1=C(C=C(C=C1)Cl)NC(=S)NCCC2=CC=C(C=C2)S(=O)...,Inactive
18232,C1CN(CCN1C2=CC=C(C=C2)[N+](=O)[O-])C(=O)CN3C(=...,Inactive


In [5]:
smiles = mlt.dfToStringMatrix(df)

18234×2 Matrix{String}:
 "COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=C(C=C3)Br)N)C(=O)C4=CC=CC=C4)OC"                                                         …  "Active"
 "COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=CC=C3)N)C(=O)C4=CC=CC=C4)OC"                                                                "Active"
 "C1CN(CCN1)C(=O)C2=C(SC(=C2)C3=CC=CC=C3)N"                                                                                         "Active"
 "C1=CC=C(C=C1)C(=O)C2=C(C(=C(N2)N)C(=O)N)C3=CC=CC=C3Br"                                                                            "Active"
 "CC1=C(C(=O)OC(C1)[C@@H](C)C2CC[C@@H]3[C@@]2(CCC4C3C[C@@H]5[C@]6([C@@]4(C(=O)C=C[C@@H]6O)C)O5)C)CO"                                "Active"
 "CC1CCC/C=C/C(=O)C2=C(CC(=O)O1)C=C(C=C2O)O"                                                                                     …  "Active"
 "CC1C2C(NC(=O)C23C(/C=C(/CCC(C(/C=C/C3=O)O)O)\\C)C=C1C)CC(C)C"                                                                   

In [6]:
vocab = mlt.dfToStringMatrix(mlt.getdf(joinpath(@__DIR__, "vocab.csv")))

71×1 Matrix{String}:
 "#"
 "("
 ")"
 "."
 "/"
 "1"
 "2"
 "3"
 "4"
 "5"
 "6"
 "7"
 "8"
 ⋮
 "[Nd+3]"
 "[O-]"
 "[OH-]"
 "[P+]"
 "[Pt+2]"
 "[Pt]"
 "[S+]"
 "[S-]"
 "[Se]"
 "[Si]"
 "[Zn+2]"
 "\\"

In [23]:
tokenizer = Dict(j => i for (i, j) in enumerate(vocab))
reverse_tokenizer = Dict(value => key for (key, value) in tokenizer)

Dict{Int64, String} with 71 entries:
  5  => "/"
  56 => "[NH3+]"
  16 => "B"
  35 => "[C@H]"
  55 => "[NH2+]"
  20 => "F"
  60 => "[Nd+3]"
  30 => "[Ba+2]"
  19 => "Cl"
  32 => "[C-]"
  49 => "[N+]"
  6  => "1"
  67 => "[S-]"
  45 => "[I-]"
  44 => "[Hg]"
  9  => "4"
  31 => "[Br-]"
  64 => "[Pt+2]"
  61 => "[O-]"
  29 => "[B-]"
  46 => "[K+]"
  57 => "[NH4+]"
  70 => "[Zn+2]"
  4  => "."
  13 => "8"
  ⋮  => ⋮

In [8]:
function readPyFile(path)
    py"""
    exec(open($(path)).read(), globals(), locals())
    """
end

readPyFile("tokenization.py")
return_tokens(str) = py"return_tokens"(str)

return_tokens (generic function with 1 method)

In [35]:
strings = [[tokenizer[j] for j in return_tokens(i)[begin]] for i in smiles[:, begin]]
activity = [i == "Active" ? 1 : 0 for i in smiles[:, end]]

@assert size(strings) == size(activity) 

In [21]:
sum(length.(strings))/length(strings)

49.81024459800373

In [33]:
test_tokens = [reverse_tokenizer[i] for i in strings[begin]]
test_string = ""
for i in test_tokens
    test_string *= i 
end

test_string

"COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=C(C=C3)Br)N)C(=O)C4=CC=CC=C4)OC"

https://fluxml.ai/Flux.jl/v0.2/examples/char-rnn.html

could potentially use smiles enumeration for data augmentation during training